In [ ]:
# Caminho do arquivo
file_path = r"xxxx"

# Lê o arquivo
with open(file_path, 'r', encoding='ISO-8859-1') as file:
    lines = file.readlines()

# Verifica se a palavra "Cód." está presente na primeira linha
if "Cód." in lines[0]:
    print("A palavra 'Cód.' está presente na primeira linha. Nada a fazer.")
else:
    # Se não está presente, apaga as 6 primeiras linhas
    with open(file_path, 'w', encoding='ISO-8859-1') as file:
        file.writelines(lines[6:])  # Escreve o restante do arquivo, ignorando as 6 primeiras linhas
    print("As 6 primeiras linhas foram apagadas.")

import csv
import os
from datetime import datetime

# Defina o caminho do arquivo de entrada e saída
base_path = r'xxxx'
txt_file_path = os.path.join(base_path, 'Indices_SAP.txt')

# Função para limpar os indicadores
def clean_indicator(indicator):
    if indicator.startswith('+'):
        return indicator[1:]  # Remove o sinal de mais
    elif indicator.startswith('-'):
        return f"negativo {indicator[1:]}"  # Adiciona um prefixo para indicar negativo
    return indicator

# Função para formatar a data
def format_date(date_str):
    try:
        date_obj = datetime.strptime(date_str, "%d.%m.%Y")
        return date_obj.strftime("%d/%m/%Y")
    except ValueError:
        return date_str  # Retorna a string original se não for uma data válida

# Passo 1: Ler o arquivo .txt e armazenar os dados
with open(txt_file_path, 'r', encoding='ISO-8859-1') as txt_file:
    lines = txt_file.readlines()

data = []
header = []
for line_index, line in enumerate(lines):
    columns = line.strip().split('\t')
    
    # Captura o cabeçalho
    if line_index == 0:
        header = [clean_indicator(col) for col in columns if col.strip() != '']
        continue
    
    # Limpar os indicadores
    cleaned_columns = [clean_indicator(col) for col in columns if col.strip() != '']
    
    # Excluir linhas com "0", "*", ou "(NAO USAR)" na coluna "Nome ind."
    if cleaned_columns and cleaned_columns[0] not in ['0', '*']:
        nome_ind = cleaned_columns[header.index("Nome Índ.")]  # Coluna "Nome ind."
        desc_ind = cleaned_columns[header.index("Descrição do Índice")]  # Coluna "Descrição do índice"
        
        # Verifica as condições de exclusão
        if "(NAO USAR)" not in nome_ind and \
           "(INDICE DESCONTINUADO) - IPA Produtos Quimicos" not in desc_ind and \
           "A0USGAAP" not in desc_ind and \
           "0,0055" not in desc_ind and \
           "JUROS USGAAP SISTEMA PETROBRAS" not in desc_ind and \
           "(DESCONTINUADO) Mat.Eletronico, Aparelho e Eq.Comunicação" not in desc_ind:
            
            # Formatar a coluna "Dt.Ini.Val" se existir
            if "Dt.Ini.Val" in header:
                ini_val_index = header.index("Dt.Ini.Val")
                if ini_val_index < len(cleaned_columns):
                    dt_val = cleaned_columns[ini_val_index].strip()
                    if dt_val:  # Verifica se não está vazia
                        cleaned_columns[ini_val_index] = format_date(dt_val)
            
            # Adicionar à lista de dados
            data.append(cleaned_columns)

# Remover as colunas indesejadas
columns_to_remove = ["Dt.Fim Ind", "Dt.Fim.Val", "Tipo", "             Tx Div Ind", "Cd BDC", "Nome ind."]
indices_to_keep = [i for i, col in enumerate(header) if col not in columns_to_remove]

# Filtrar as colunas nos dados, garantindo que as linhas tenham o número correto de colunas
filtered_data = []
for row in data:
    if "Dt.Ini.Val" in header:
        ini_val_index = header.index("Dt.Ini.Val")
        if ini_val_index < len(row):
            # Pula esta linha se contiver anos indesejados
            if any(year in row[ini_val_index] for year in ["2094", "2095", "2096", "2097", "2098", "2099"]):
                continue
            # Pula a linha se "Dt.Ini.Val" estiver vazia
            if row[ini_val_index].strip() == '':
                continue  # Pula esta linha se "Dt.Ini.Val" estiver vazia

    # Filtrar as colunas
    filtered_row = [row[i] for i in indices_to_keep if i < len(row)]
    filtered_data.append(filtered_row)

# Trocar '.' por '/' na coluna "Dt.In.Ind" fora do loop
if "Dt.In.Ind" in header:
    dt_ind_index = header.index("Dt.In.Ind")
    for row in filtered_data:
        if dt_ind_index < len(row):
            # Trocar '.' por '/' na coluna "Dt.In.Ind"
            row[dt_ind_index] = row[dt_ind_index].replace('.', '/')

# Passo 2: Salvar os dados completos iniciais em um novo arquivo CSV
initial_data_file_path = os.path.join(base_path, 'Indices_SAP.csv')

with open(initial_data_file_path, 'w', newline='', encoding='utf-8-sig') as initial_file:
    writer = csv.writer(initial_file, delimiter=';')  # Aqui você define o delimitador
    writer.writerow([col for col in header if col not in columns_to_remove])  # Grava o cabeçalho sem as colunas removidas
    writer.writerows(filtered_data)

print(f"Dados completos iniciais salvos em {initial_data_file_path}")


import csv
import os

# Defina o caminho do arquivo CSV
base_path = r'xxxx'
csv_file_path = os.path.join(base_path, 'Indices_SAP.csv')

# Lista de indicadores
indicadores = [
    "01M", "01U11", "12PAV", "29.09", "36PAV", "ABH", "ACE", "ACR", "ACVES", "AIRCR",
    "AIRTR", "ALB", "ALC", "ALD", "ALI", "ALS", "AMO", "AMT", "ANITR", "ARJ", "ASG",
    "ATL", "ATR", "AVG", "BCD", "BCT", "BDT1", "BID", "BLSHE", "BLSIB", "BLSIN", "BLSME",
    "BLSPL", "BLSST", "BMO", "BMT", "BNA", "BND1", "BPT", "BRAS", "BSPPI", "BTA", "BTB",
    "BTC", "BTD", "BTE", "BTG", "BTH", "BTI", "BTL", "C28", "CACR", "CAL", "CAP", "CAS",
    "CBH", "CCA", "CCAE", "CCGM", "CCGO", "CCM", "CCSL", "CCV", "CEP", "CEPEA", "CFE",
    "CH", "CHUUS", "CNCC", "CND", "COP1", "COT", "CPA", "CPICH", "CPIUS", "CPV", "CRJ",
    "CRUEU", "CRUSL", "CRUST", "CSC", "DAB", "DBA", "DBH", "DI12P", "DI36P", "DIB",
    "DICUB", "DIE", "DIMP", "DINOR", "DIP", "DLT", "DPI", "DRC", "DRI", "DRJ", "DRPI",
    "DRS", "DTB", "DTU", "DUD", "ECR", "EHH", "ELE", "EMI", "ENC", "EQN", "ESM", "EST",
    "EUC28", "EURME", "EXP", "FAD", "FDE", "FF1", "FF2", "FF3", "FF4", "FF5", "FFR",
    "FLM", "FMO", "FTL", "FTM", "FTZ", "GAB", "GAP", "GAS", "GASBU", "GAV", "GBA", "GBH",
    "GNA", "GOIA", "GVE", "HDPE", "HICP", "HICPA", "HPF", "HRJ", "IAC", "IAG", "IAM",
    "IBH", "ICA", "ICCBT", "ICCV", "ICF", "ICFET", "ICISL", "ICP", "IDTR", "IEN", 
    "IFFAS", "IGPS", "ILSP", "IMD", "IMI", "IMO1", "IMOL", "IMRJ", "IMT", "INCTG", 
    "INPCT", "IOHMD", "IPAAB", "IPAAC", "IPAAE", "IPAAI", "IPAAS", "IPAAV", "IPABA", 
    "IPABB", "IPABD", "IPABH", "IPACC", "IPACE", "IPACJ", "IPACR", "IPACT", "IPADP", 
    "IPAES", "IPAFA", "IPAFE", "IPAGA", "IPAIE", "IPAMO", "IPAPC", "IPAPF", "IPAPS", 
    "IPARA", "IPARJ", "IPART", "IPARV", "IPASD", "IPASS", "IPATA", "IPATC", "IPATE", 
    "IPATM", "IPATR", "IPAVI", "IPBCL", "IPBH", "IPBPC", "IPC-M", "IPC15", "IPCAG", 
    "IPCAR", "IPCAS", "IPCF", "IPCIT", "IPCMO", "IPCR", "IPCSA", "IPCTR", "IPED", 
    "IPEP", "IPETI", "IPFAC", "IPFEP", "IPMTN", "IPOAV", "IPPAM", "IPPAS", "IPPBF", 
    "IPPDP", "IPPFE", "IPPLF", "IPPMN", "IPPMV", "IPPOC", "IPPSE", "IPR", "IPRRM", 
    "IPTAC", "IPTAS", "IPTCA", "IPTPL", "IPTRH", "IPVAC", "IPVTR", "IRT", "ISC", 
    "ISER", "ISMG", "ISRE", "ISVS", "ITM", "ITML", "ITT", "IVSA", "IVSS", "JLP", 
    "JLP1", "JMO", "JPT", "LAC", "LCSP", "LEF", "LEX", "LIB", "LIR", "LIT", "LMECH", 
    "LMECO", "LUB", "MAA", "MAC", "MAL", "MAM", "MAM1", "MAMEL", "MATP", "MBT", 
    "MCG", "MCL", "MCT", "MEC", "MED", "MEE", "MELG", "MEQ", "MET", "MFCO", "MGE", 
    "MMA", "MMO", "MMT", "MNA", "MNF", "MOA", "MOB", "MOD", "MOE", "MON", "MOU", 
    "MPB", "MPS", "MPT", "MTC", "MTEC", "MTL1", "MTM", "MTO", "MTT", "MTV", "MVO", 
    "MVT", "NOU", "NSPPI", "NSRPP", "OAT", "OBRPT", "OCM", "ODI", "OECD1", "OECDT", 
    "OFG", "OHA", "OHAC", "OHACP", "OHC", "OHCC", "OHCE", "OHCMC", "OHCP", "OHE", 
    "OHEC", "OHEE", "OHEQP", "OHFM", "OHLT", "OHMB", "OHPFF", "OHPP", "OHPSD", 
    "OHRCH", "OHRSB", "OROAE", "OUS", "PAA", "PAC", "PAF", "PAG", "PAG1", "PAL", 
    "PAO", "PAR", "PBO", "PCU13", "PCU15", "PCU21", "PCUZ2", "PEP", "PIN", "PIQ", 
    "PMO", "PMP1", "PMT", "PNE", "POL", "PPDI", "PPOG", "PRF", "PRJ", "PSV", "PTL", 
    "QAR1", "QAV", "QAV09", "QAV27", "QAVBR", "QAVSD", "QMP", "REE", "RGS", "RJM", 
    "ROUPA", "RRJ", "SAU", "SERES", "SETBA", "SETSP", "SEVES", "SFR", "SIMET", 
    "SIND", "SMR", "SPT", "SRJ", "SRO", "STL", "STRAP", "SVBA", "SVBAR", "SVITA", 
    "SVMG", "SVP", "SVPEC", "SVR", "SVS", "SVSC", "TAL", "TAMO", "TEB", "TEC", 
    "TEG", "TEL", "TEN", "TENIS", "TFN", "TFS", "TMRF", "TOT", "TRD", "TRR", "TSL", 
    "TVC", "TVC1", "TXA", "TXD", "TXF", "UFM", "UKBM", "UKME", "UNM", "URB", "URI", 
    "URIP", "URO", "URT", "USC", "USD1", "VER", "VES", "VIGRJ", "VPT", "VRJ", "VRJ1", 
    "VSAM", "VSB", "VSG", "VSN", "VSP", "VSPA", "VSSP", "WPU02", "WPU30", "]TRM"
]

# Função para substituir '.' por '/'
def replace_dot_with_slash(value):
    return value.replace('.', '/')

# Função para substituir "negativo" por "-"
def replace_negativo(value):
    return value.replace('negativo ', '-')

# Função para verificar se a data está acima do ano vigente
def is_date_valid(date_str):
    try:
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        return date_obj.year <= datetime.now().year
    except ValueError:
        return False

# Passo 1: Ler o arquivo CSV
with open(csv_file_path, 'r', encoding='utf-8-sig') as csv_file:
    reader = csv.reader(csv_file, delimiter=';')
    header = next(reader)  # Ler o cabeçalho
    header = [col.strip() for col in header]  # Remover espaços em branco

    # Encontrar índices das colunas relevantes
    cod_index = header.index("Cód.")
    dt_ini_val_index = header.index("Dt.Ini.Val")
    dt_in_ind_index = header.index("Dt.In.Ind")
    vl_divulgado_index = header.index("Vl.Divulgado")
    desc_ind_index = header.index("Descrição do Índice")

    data = []

    # Processar cada linha
    for row in reader:
        if len(row) <= dt_in_ind_index:
            print(f"Linha inválida, ignorando: {row}")
            continue  # Ignora linhas com número insuficiente de colunas

        # Substituir '.' por '/' nas colunas de data
        row[dt_in_ind_index] = replace_dot_with_slash(row[dt_in_ind_index])

        # Adicionar nova coluna "Data"
        new_data = row[dt_ini_val_index] if row[cod_index] in indicadores else row[dt_in_ind_index]
        row.append(new_data)

        # Inserir "SAP - " antes do código
        row[cod_index] = f"SAP - {row[cod_index]}"
        data.append(row)

# Passo 2: Remover as colunas "Dt.Ini.Val" e "Dt.In.Ind" antes de salvar
columns_to_remove = [dt_ini_val_index, dt_in_ind_index]
filtered_data = []

for row in data:
    filtered_row = [val for i, val in enumerate(row) if i not in columns_to_remove]

    # Verificar se a data é válida
    if is_date_valid(filtered_row[-1]):  # A última coluna é a nova "Data"
        # Substituir "negativo" por "-" na coluna "Vl.Divulgado"
        vl_divulgado_value = filtered_row[vl_divulgado_index]
        filtered_row[vl_divulgado_index] = replace_negativo(vl_divulgado_value)

        # Criar o novo valor para "nome_completo"
        codigo_value = row[cod_index]  # Pegar o código original
        descricao_ind_value = row[desc_ind_index]  # Manter o valor original da descrição
        nome_completo_value = f"{codigo_value} - {descricao_ind_value}"  # Criar o novo valor
        
        # Substituir o valor existente de "Descrição do Índice" por "nome_completo"
        filtered_row[desc_ind_index] = nome_completo_value  # Substitui o antigo pelo novo

        filtered_data.append(filtered_row)

# Passo 3: Salvar os dados modificados de volta no arquivo CSV
with open(csv_file_path, 'w', newline='', encoding='utf-8-sig') as csv_file:
    writer = csv.writer(csv_file, delimiter=';')
    # Escrever o cabeçalho renomeando colunas
    new_header = [col if col != "Cód." else "codigo" for col in header if col not in ["Dt.Ini.Val", "Dt.In.Ind", "Descrição do Índice"]]
    new_header.insert(desc_ind_index, "nome_completo")  # Adiciona a nova coluna "nome_completo"
    new_header += ["data"]  # Adiciona a nova coluna "Data"
    
    writer.writerow(new_header)
    writer.writerows(filtered_data)

print(f"Arquivo atualizado com sucesso em {csv_file_path}")



import csv
import os

# Defina o caminho do arquivo CSV
base_path = r'xxxx'
csv_file_path = os.path.join(base_path, 'Indices_SAP.csv')

# Função para remover a coluna, renomear outra coluna e adicionar uma nova coluna
def modify_columns_and_save(csv_file_path, column_to_remove, old_column_name, new_column_name):
    with open(csv_file_path, 'r', encoding='utf-8-sig') as csv_file:
        reader = csv.reader(csv_file, delimiter=';')
        header = next(reader)  # Ler o cabeçalho
        header = [col.strip() for col in header]  # Remover espaços em branco

        # Encontrar o índice da coluna a ser removida
        if column_to_remove in header:
            column_index = header.index(column_to_remove)
        else:
            print(f"Coluna '{column_to_remove}' não encontrada.")
            return

        # Encontrar o índice da coluna a ser renomeada
        if old_column_name in header:
            old_column_index = header.index(old_column_name)
        else:
            print(f"Coluna '{old_column_name}' não encontrada.")
            return

        # Filtrar o cabeçalho
        new_header = [col if col != old_column_name else new_column_name for col in header if col != column_to_remove]
        new_header.append("fonte")  # Adiciona nova coluna "fonte"

        filtered_data = []
        for row in reader:
            filtered_row = [val for i, val in enumerate(row) if i != column_index]
            # Ajustar a renomeação
            filtered_row[old_column_index - (1 if column_index < old_column_index else 0)] = row[old_column_index]
            filtered_row.append("SAP")  # Adiciona "SAP" na nova coluna
            filtered_data.append(filtered_row)

    # Salvar os dados modificados de volta no arquivo CSV
    with open(csv_file_path, 'w', newline='', encoding='utf-8-sig') as csv_file:
        writer = csv.writer(csv_file, delimiter=';')
        writer.writerow(new_header)
        writer.writerows(filtered_data)

    print(f"Coluna '{column_to_remove}' removida, '{old_column_name}' renomeada para '{new_column_name}', e coluna 'fonte' adicionada com sucesso em {csv_file_path}")

# Chame a função para remover a coluna "Nome Índ." e renomear "Vl.Divulgado" para "valor"
modify_columns_and_save(csv_file_path, "Nome Índ.", "Vl.Divulgado", "valor")

import csv
import os

# Defina os caminhos dos arquivos CSV
base_path = r'xxxx'
indices_file_path = os.path.join(base_path, 'Indices_SAP.csv')
moedas_file_path = os.path.join(base_path, 'moedas_relacionadas.csv')

# Função para adicionar a coluna "moedas relacionadas" ao arquivo de índices
def add_moedas_relacionadas(indices_file_path, moedas_file_path):
    # Ler as moedas relacionadas
    moedas_dict = {}
    with open(moedas_file_path, 'r', encoding='utf-8-sig') as moedas_file:
        reader = csv.reader(moedas_file, delimiter=';')
        header = next(reader)  # Ler o cabeçalho
        header = [col.strip() for col in header]  # Remover espaços em branco

        # Criar um dicionário para moedas relacionadas
        for row in reader:
            if len(row) == 2:  # Verifica se a linha tem duas colunas
                codigo = row[0].strip()
                moedas_relacionadas = row[1].strip()
                moedas_dict[codigo] = moedas_relacionadas

    # Ler o arquivo de índices e adicionar a coluna "moedas relacionadas"
    with open(indices_file_path, 'r', encoding='utf-8-sig') as indices_file:
        reader = csv.reader(indices_file, delimiter=';')
        header = next(reader)  # Ler o cabeçalho
        header = [col.strip() for col in header]  # Remover espaços em branco

        # Adicionar nova coluna ao cabeçalho
        new_header = header + ["moedas relacionadas"]
        filtered_data = []

        for row in reader:
            codigo = row[header.index("codigo")].strip()  # Obter o código
            moedas_relacionadas = moedas_dict.get(codigo, "BRL")  # Pega "BRL" se não encontrado
            filtered_row = row + [moedas_relacionadas]  # Adicionar nova coluna
            filtered_data.append(filtered_row)

    # Salvar os dados modificados de volta no arquivo CSV
    with open(indices_file_path, 'w', newline='', encoding='utf-8-sig') as indices_file:
        writer = csv.writer(indices_file, delimiter=';')
        writer.writerow(new_header)  # Escrever o novo cabeçalho
        writer.writerows(filtered_data)  # Escrever os dados

    print(f"Coluna 'moedas relacionadas' adicionada com sucesso em {indices_file_path}")

# Chame a função
add_moedas_relacionadas(indices_file_path, moedas_file_path)

import pandas as pd
import os
from datetime import datetime

# Caminho do arquivo original
caminho_arquivo_original = r"xxxx"

# Lê o arquivo CSV original com o separador correto
df = pd.read_csv(caminho_arquivo_original, sep=';', encoding='utf-8-sig')

# Remove espaços em branco dos nomes das colunas
df.columns = df.columns.str.strip()

# Imprime os nomes das colunas para verificação
print("Nomes das colunas:")
print(df.columns.tolist())

# Filtra linhas onde a coluna "data" é nula
df = df.dropna(subset=['data'])

# Agrupa os dados e obtém a data mais recente para cada código e nome completo
resultado_final = df.groupby(['codigo', 'nome_completo']).agg(
    Data_do_ultimo_dado=('data', 'max')  # Usa a coluna "data"
).reset_index()

# Renomeia as colunas
resultado_final.columns = ['codigo', 'nome_completo', 'Data_do_ultimo_dado']

# Adiciona a data de atualização da base (data atual)
data_atualizacao_base = datetime.now().strftime('%Y-%m-%d')
resultado_final['Data_de_atualizacao_da_base'] = data_atualizacao_base

# Define o caminho do novo arquivo CSV
caminho_novo_arquivo = os.path.join(
    os.path.dirname(caminho_arquivo_original), 
    'Indices_SAP_resultado.csv'
)

# Salva o novo arquivo CSV com a codificação utf-8-sig
resultado_final.to_csv(caminho_novo_arquivo, index=False, sep=";", encoding='utf-8-sig')

print(f'Arquivo salvo em: {caminho_novo_arquivo}')




A palavra 'Cód.' está presente na primeira linha. Nada a fazer.
Dados completos iniciais salvos em C:\Users\FNEQ\PETROBRAS\Integração de Dados - SUB ORC - Scripts\Scripts\SAP\Indices_SAP.csv
Linha inválida, ignorando: ['DFB01', 'RISER - RB', 'RISER - RB Carcaça: 304L/ Camada de Poliamida', '       100,00000000']
Linha inválida, ignorando: ['QAV20', 'QAV20', 'Querosene de Aviação - Sudeste Preço Dia 20', '         1,57910000', '01/01/1900']
Arquivo atualizado com sucesso em C:\Users\FNEQ\PETROBRAS\Integração de Dados - SUB ORC - Scripts\Scripts\SAP\Indices_SAP.csv
Coluna 'Nome Índ.' removida, 'Vl.Divulgado' renomeada para 'valor', e coluna 'fonte' adicionada com sucesso em C:\Users\FNEQ\PETROBRAS\Integração de Dados - SUB ORC - Scripts\Scripts\SAP\Indices_SAP.csv
Coluna 'moedas relacionadas' adicionada com sucesso em C:\Users\FNEQ\PETROBRAS\Integração de Dados - SUB ORC - Scripts\Scripts\SAP\Indices_SAP.csv
Nomes das colunas:
['codigo', 'nome_completo', 'valor', 'data', 'fonte', 'moedas